In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/BETTER30.csv')
df

,CONVERSATION_ID,CONVERSATION_STEP,TEXT,CONTEXT,LABEL,FEATURES,ANNOTATIONS
0,6,1,"Good morning, this is [Your Name]'s personal a...",Standard opening exchange,neutral,NaN,NaN
1,6,2,"Hello, my name is Jamie. I'm interested in vol...",Encourages the caller's interest,neutral,"welcoming, positive_tone",NaN
2,6,3,"Yes, I'm really passionate about environmental...",Reinforces anyone can volunteer,neutral,inclusive,NaN
3,6,4,"Great, how do I sign up, and where can I find ...",Demonstrates flexibility,neutral,"helpful_tone, offers_options",NaN
4,6,5,"Could you send me the link, please? And my ema...",Fulfills caller's request quickly,neutral,prompt_action,NaN
...,...,...,...,...,...,...,...
645,68,5,Would it be possible to receive some literatur...,"""Assistant requests official documentation fo...",scam_response,"""literature_request","decision_making_review"""
646,68,6,We can certainly provide information on our wo...,"""Scammer stresses the urgency of the situation.""",scam,"""immediate_action_urgency","urgency_stress"""
647,68,7,"I understand the urgency, but our policy is to...","""Assistant reiterates the policy on verifying...",scam_response,"""contribution_policy_reiteration","information_for_review_request"""
648,68,8,I'll do my best to get you the information qui...,"""Scammer promises to send information and hig...",scam,"""information_sending_promise","support_impact_highlight"""


In [ ]:
val=df[df['LABEL']=='neutral']['TEXT']

In [ ]:
val.iloc[0]

"Good morning, this is [Your Name]'s personal assistant. How can I help you today?"

In [ ]:
category_counts = df['LABEL'].value_counts()

# Identify categories to keep (count >= 2)
categories_to_keep = category_counts[category_counts >= 2].index

# Filter the DataFrame
df = df[df['LABEL'].isin(categories_to_keep)]


In [ ]:
df.drop(columns=['CONVERSATION_ID','CONVERSATION_STEP','CONTEXT','FEATURES','ANNOTATIONS'],inplace=True)

In [ ]:
df['LABEL'].value_counts()

In [ ]:
df = df.rename(columns={'LABEL': 'labels'})
df = df.rename(columns={'TEXT': 'text'})


In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['labels']=le.fit_transform(df['labels'])

In [ ]:
df['labels'].value_counts()

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split


train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['labels'], random_state=42)


train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=13)

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    # evaluation_strategy="epoch",
    save_total_limit=1,
    fp16=True,
    report_to="none",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


trainer.train()


trainer.save_model('./roberta-audio-model')
tokenizer.save_pretrained('./roberta-audio-model')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/513 [00:00<?, ? examples/s]

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

Step,Training Loss


('./roberta-audio-model/tokenizer_config.json',
 './roberta-audio-model/special_tokens_map.json',
 './roberta-audio-model/vocab.json',
 './roberta-audio-model/merges.txt',
 './roberta-audio-model/added_tokens.json')

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

model = RobertaForSequenceClassification.from_pretrained('./roberta-audio-model')
tokenizer = RobertaTokenizer.from_pretrained('./roberta-audio-model')

model.save_pretrained('./roberta-audio-model-inference')
tokenizer.save_pretrained('./roberta-audio-model-inference')


('./roberta-audio-model-inference/tokenizer_config.json',
 './roberta-audio-model-inference/special_tokens_map.json',
 './roberta-audio-model-inference/vocab.json',
 './roberta-audio-model-inference/merges.txt',
 './roberta-audio-model-inference/added_tokens.json')

In [ ]:
import torch

from IPython.display import display
from transformers import RobertaTokenizer, RobertaForSequenceClassification


tokenizer = RobertaTokenizer.from_pretrained('/content/roberta-audio-model-inference')
model = RobertaForSequenceClassification.from_pretrained('/content/roberta-audio-model-inference')

device = torch.device('cpu')
model.to(device)
model.eval()

def get_prediction(input_text):
    inputs = tokenizer([input_text], return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    predicted_label = predictions.item()
    result=le.inverse_transform([predicted_label])
    return result

get_prediction('''Good morning, this is [Your Name]'s personal assistant. How can I help you today?''')

array(['scam'], dtype=object)

In [ ]:
from google.colab import files

In [ ]:
import pickle

In [ ]:
pickle.dump(le,open('encoder.pkl','wb'))

In [ ]:
!zip -r /content/folder_name.zip /content/roberta-audio-model-inference


  adding: content/roberta-audio-model-inference/ (stored 0%)
  adding: content/roberta-audio-model-inference/config.json (deflated 60%)
  adding: content/roberta-audio-model-inference/vocab.json (deflated 68%)
  adding: content/roberta-audio-model-inference/model.safetensors (deflated 17%)
  adding: content/roberta-audio-model-inference/merges.txt (deflated 53%)
  adding: content/roberta-audio-model-inference/tokenizer_config.json (deflated 76%)
  adding: content/roberta-audio-model-inference/special_tokens_map.json (deflated 84%)


In [ ]:
files.download('/content/folder_name.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>